# Experiment 001: Baseline from Pre-optimized Submission

Download and validate the pre-optimized submission from GitHub, score it locally.

In [1]:
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree
import warnings
warnings.filterwarnings('ignore')

getcontext().prec = 30

print('Libraries loaded')

Libraries loaded


In [2]:
# ChristmasTree class
class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))
        
        # Tree dimensions
        trunk_w, trunk_h = Decimal('0.15'), Decimal('0.2')
        base_w, mid_w, top_w = Decimal('0.7'), Decimal('0.4'), Decimal('0.25')
        tip_y, tier_1_y, tier_2_y = Decimal('0.8'), Decimal('0.5'), Decimal('0.25')
        
        # 15-vertex polygon
        initial_polygon = Polygon([
            (0, float(tip_y)), (float(top_w/2), float(tier_1_y)), (float(top_w/4), float(tier_1_y)),
            (float(mid_w/2), float(tier_2_y)), (float(mid_w/4), float(tier_2_y)), (float(base_w/2), 0),
            (float(trunk_w/2), 0), (float(trunk_w/2), float(-trunk_h)),
            (float(-trunk_w/2), float(-trunk_h)), (float(-trunk_w/2), 0), (float(-base_w/2), 0),
            (float(-mid_w/4), float(tier_2_y)), (float(-mid_w/2), float(tier_2_y)),
            (float(-top_w/4), float(tier_1_y)), (float(-top_w/2), float(tier_1_y))
        ])
        
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated, xoff=float(self.center_x), yoff=float(self.center_y))

print('ChristmasTree class defined')

ChristmasTree class defined


In [3]:
# Scoring functions
def get_tree_list_side_length(tree_list):
    all_polygons = [t.polygon for t in tree_list]
    bounds = unary_union(all_polygons).bounds
    return Decimal(str(max(bounds[2] - bounds[0], bounds[3] - bounds[1])))

def get_total_score(dict_of_side_length):
    score = Decimal('0')
    for k, v in dict_of_side_length.items():
        score += v ** 2 / Decimal(str(k))
    return score

# Overlap detection
def has_overlap(trees):
    if len(trees) <= 1:
        return False
    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx != i and poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                intersection = poly.intersection(polygons[idx])
                if intersection.area > 1e-10:
                    return True
    return False

print('Scoring and overlap functions defined')

Scoring and overlap functions defined


In [4]:
# Load submission
def load_submission(path):
    df = pd.read_csv(path)
    
    def parse_value(val):
        if isinstance(val, str) and val.startswith('s'):
            return val[1:]
        return str(val)
    
    df['x'] = df['x'].apply(parse_value)
    df['y'] = df['y'].apply(parse_value)
    df['deg'] = df['deg'].apply(parse_value)
    
    configs = {}
    for _, row in df.iterrows():
        parts = row['id'].split('_')
        n = int(parts[0])
        if n not in configs:
            configs[n] = []
        configs[n].append(ChristmasTree(row['x'], row['y'], row['deg']))
    
    return configs

print('Load function defined')

Load function defined


In [5]:
# Load the pre-optimized submission
configs = load_submission('/home/code/experiments/001_baseline/preoptimized_submission.csv')
print(f'Loaded {len(configs)} configurations (n=1 to n={max(configs.keys())})')

Loaded 200 configurations (n=1 to n=200)


In [6]:
# Calculate side lengths and total score
side_lengths = {}
for n in sorted(configs.keys()):
    trees = configs[n]
    side_length = get_tree_list_side_length(trees)
    side_lengths[n] = side_length
    if n <= 10 or n >= 195:
        print(f'n={n}: side_length={float(side_length):.6f}, trees={len(trees)}')

total_score = get_total_score(side_lengths)
print(f'\nTotal Score: {float(total_score):.6f}')

n=1: side_length=0.813173, trees=1
n=2: side_length=0.949504, trees=2
n=3: side_length=1.142031, trees=3
n=4: side_length=1.290806, trees=4
n=5: side_length=1.443692, trees=5
n=6: side_length=1.548438, trees=6
n=7: side_length=1.673104, trees=7
n=8: side_length=1.755921, trees=8
n=9: side_length=1.867280, trees=9
n=10: side_length=1.940696, trees=10


n=195: side_length=8.079853, trees=195
n=196: side_length=8.088244, trees=196
n=197: side_length=8.184293, trees=197
n=198: side_length=8.202895, trees=198
n=199: side_length=8.207855, trees=199
n=200: side_length=8.220566, trees=200

Total Score: 70.743774


In [7]:
# Check for overlaps in all configurations
print('Checking for overlaps...')
overlapping_configs = []
for n in sorted(configs.keys()):
    if has_overlap(configs[n]):
        overlapping_configs.append(n)
        if len(overlapping_configs) <= 10:
            print(f'  Overlap detected in n={n}')

if overlapping_configs:
    print(f'\nTotal configurations with overlaps: {len(overlapping_configs)}')
else:
    print('\nNo overlaps detected! Submission is valid.')

Checking for overlaps...



No overlaps detected! Submission is valid.


In [ ]:
# Copy to submission folder
import shutil
import json

shutil.copy('/home/code/experiments/001_baseline/preoptimized_submission.csv', '/home/submission/submission.csv')
print('Copied to /home/submission/submission.csv')

with open('/home/code/experiments/001_baseline/metrics.json', 'w') as f:
    json.dump({'cv_score': float(total_score), 'overlapping_configs': overlapping_configs}, f)
print(f'\nFinal Score: {float(total_score):.6f}')